# Module 8: LDA From Scatch

We develop an LDA topic modeler using collapsed Gibbs sample as described by [Griffiths and Steyvers (2004)](https://collab.its.virginia.edu/access/content/group/b9e58ce7-0f44-48fe-9861-b7a7657f551a/Articles/sciencetopics.pdf).

# Setup

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from nltk.corpus import stopwords 

# Functions

## Convert F1 Corpus 

We convert the F1 corpus into TOKEN and VOCAB tables.

In [ ]:
def convert_corpus(docs):
    
    global TOKEN, VOCAB
    
    # Convert docs into tokens
    stop_words = set(stopwords.words('english')) 
    tokens = []
    for i, doc in enumerate(docs):
        for j, token in enumerate(doc.split()):
            term_str = re.sub(r'[\W_]+', '', token).lower()
            if term_str not in stop_words:
                tokens.append((i, j, term_str))
    TOKEN = pd.DataFrame(tokens, columns=['doc_id','token_num','term_str'])\
        .set_index(['doc_id','token_num'])
    
    # Extract vocabulary
    VOCAB = TOKEN.term_str.value_counts().to_frame().reset_index()
    VOCAB.columns = ['term_str', 'n']
    VOCAB.index.name = 'term_id'
    TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)
    

## Initialize the Model

We convert TOKEN into a BOW, randomly assign topic labels, and generate PHI and THETA tables.

In [ ]:
def init_model():
    
    global n_topics, n_docs, n_words, TOKEN, BOW, TOPIC, THETA, PHI, topic_names

    # Extract BOW from TOKEN
    BOW = TOKEN.groupby(['doc_id', 'term_id']).term_id.count()\
        .to_frame().rename(columns={'term_id':'n'})
    
    # Normalize n
    # May want normalize n to binary or log form
    
    # Create TOPIC table
    TOPIC = pd.DataFrame(index=range(n_topics))
    TOPIC.index.name = 'topic_id'
    topic_names = TOPIC.index.tolist()
    
    # Randomly assign topics to words (word = term in BOW)
    BOW['topic_id'] = TOPIC.sample(BOW.shape[0], replace=True).index
    
    # Generete topic-doc count matrix
    THETA = BOW.groupby(['topic_id', 'doc_id']).n.sum()\
        .unstack().fillna(0).astype('int')

    # Generate term-topic matrix (aka word-topic)
    PHI = BOW.groupby(['term_id', 'topic_id']).n.sum()\
        .unstack().fillna(0).astype('int')        

    # Get doc and word counts
    n_docs = THETA.shape[1]
    n_words = PHI.shape[0]
    

## Gibbs Sampler

We sample each document and word combination in the BOW table. In each case,
we are looking for two values:

* the topic with which a word has been most frequently labeled
* the topic with which the document has the most labeled words

We combine these values in order to align the label of the current word with the rest of the data.\
If a the topic is highly associated with both the word and the document, then that topic will get a high value.

Note that all that is going on here is a sorting operation -- the random assignment does not predict anything.\
Instead, we are just gathering words under topics and topics under documents.

In [ ]:
def gibbs_sample(d, w):
    
    global n_topics, n_docs, n_words, BOW, PHI, THETA, alpha, beta, topic_names
    
    # Get current topic for word in doc
    z1 = BOW.at[(d, w), 'topic_id']
    
    # Get the number of tokens for the word
    n = BOW.at[(d, w), 'n']
    
    # Remove current assignment from the counts
    PHI.at[w, z1] -= n
    THETA.at[z1, d] -= n

    # Sample from the two count matrices
    weights = np.zeros(n_topics)
    for t in topic_names:
        
        # How much this word is currently labeled by each topic
        P = (PHI.at[w, t] + alpha) / (PHI[t].sum() + n_words * alpha)
        
        # How much each topic currently appears in this document
        T = (THETA.at[t, d] + beta) / (THETA[d].sum() + n_topics * beta)
        
        # The combined value of P and T.
        weights[t] = P * T
            
    # Alternate. These operations marginalize over topics; no need to iterate
    # P = (PHI.loc[w] + alpha) / (PHI.sum() + n_words * alpha)
    # T = (THETA[d] + beta) / (THETA[d].sum() + n_topics * beta)
    # weights = (P * T).values

    # Draw new topic value from weighted list of topics
    pwgt = weights / weights.sum()
    z2 = np.random.choice(topic_names, p=pwgt)
    
    # Apply the value of the word to the topic  
    PHI.at[w, z2] += n
    THETA.at[z2, d] += n
        
    # Update the topic assignment
    BOW.at[(d , w), 'topic_id'] = z2

## Show Results

We present a summary view of  words associated with topics and topics associated with documents.

In [ ]:
def show_results():
    
    global VOCAB, TOPIC, THETA, PHI, topic_names, docs
        
    P = PHI[topic_names] / PHI[topic_names].sum()
    
    P['term_str'] = VOCAB.term_str 
    for t in topic_names:
        top_terms = P.sort_values(t, ascending=False).head(10)[[t, 'term_str']]
        print('-' * 80)
        print(top_terms)
        # Could add a string to TOPIC.loc[t, 'top_terms'] to demonstrateuse of tables to capture information.
    
    print('-' * 80)
    
    DOC = THETA.T
    DOC['doc'] = docs
    print(DOC.head(1000))
    
    print('-' * 80)

# Demonstration

We use a toy example to see if the method works.\
Because our codd is not vert efficient, we just 

## Data

A small F1 corpus.

In [ ]:
docs = """
I ate a banana and a spinach smoothie for breakfast.
I like to eat broccoli and bananas.
Chinchillas and kittens are cute.
My sister adopted a kitten yesterday.
Look at this cute hamster munching on a piece of broccoli.
""".split("\n")[1:-1]

## Parameters

In [ ]:
n_topics = 2
n_iters = 5000
alpha = .01
beta = .1

## Process

In [ ]:
%%time
convert_corpus(docs)
init_model()
for i in tqdm(range(n_iters)):
    BOW.apply(lambda x: gibbs_sample(x.name[0], x.name[1]), 1) 

100%|██████████| 5000/5000 [01:21<00:00, 61.04it/s]

CPU times: user 1min 20s, sys: 4.74 s, total: 1min 25s
Wall time: 1min 21s


In [ ]:
show_results()

--------------------------------------------------------------------------------
topic_id         0     term_str
term_id                        
0         0.222222         cute
2         0.111111      adopted
3         0.111111      hamster
4         0.111111  chinchillas
18        0.111111         look
17        0.111111    yesterday
15        0.111111      kittens
11        0.111111       kitten
12        0.000000         like
16        0.000000       sister
--------------------------------------------------------------------------------
topic_id         1  term_str
term_id                     
1         0.153846  broccoli
10        0.076923    banana
8         0.076923     piece
16        0.076923    sister
14        0.076923  smoothie
13        0.076923       ate
12        0.076923      like
9         0.076923  munching
19        0.076923       eat
7         0.076923   spinach
--------------------------------------------------------------------------------
topic_id  0  1           

In [ ]:
(PHI / PHI.sum()).style.background_gradient()

topic_id,0,1
term_id,,
0,0.222222,0
1,0,0.153846
2,0.111111,0
3,0.111111,0
4,0.111111,0
5,0,0.0769231
6,0,0.0769231
7,0,0.0769231
8,0,0.0769231


In [ ]:
(THETA / THETA.sum()).T.style.background_gradient()

topic_id,0,1
doc_id,,
0,0,1
1,0,1
2,1,0
3,0.75,0.25
4,0.5,0.5
